# Two Springs and Harmonic Forcing

In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
from IPython.display import HTML
from scipy.signal import find_peaks
from typing import NamedTuple
from scipy.fft import fft, fftfreq, ifft, fftshift
from scipy import signal

In [ ]:

def draw_spring(x0, y0, x1, y1, coils=10, amplitude=0.1, linewidth=2, color='blue'):
    """Draws a spring between (x0, y0) and (x1, y1)."""
    num_points=500

    dx = x1 - x0
    dy = y1 - y0
    length = np.sqrt(dx**2 + dy**2)
    angle = np.arctan2(dy, dx)

    x_line = np.linspace(x0, x1, num_points)
    y_line = np.linspace(y0, y1, num_points)

    coil_x = amplitude * np.cos(np.linspace(0, coils * 2 * np.pi, num_points))
    coil_y = amplitude * np.sin(np.linspace(0, coils * 2 * np.pi, num_points))

    x_spring = x_line + coil_x * np.cos(angle + np.pi/2)
    y_spring = y_line + coil_y * np.sin(angle + np.pi/2)

    return x_spring, y_spring  # Return the spring coordinates


# Animation setup
def SetupAnimation():
    fig, ax = plt.subplots()
    ax.plot([0.0,0.0],[-0.25,0.25],color='red',linewidth=4)
    line, = ax.plot([], [], lw=2, color='blue')  # Initialize an empty line object
    line2, = ax.plot([], [], lw=2, color='red')  # Initialize an empty line object

    center_x=0
    center_y=0
    radius = 0.1

    circle = patches.Circle((center_x, center_y), radius, color='blue', fill=True)
    ax.add_artist(circle)

    ax.set_xlim(-6, 45) # Set appropriate limits for x
    ax.set_ylim(-1, 1)    # Set appropriate limits for y. Adjust as needed.
    #ax.set_aspect('equal') # Important for proper spring visualization
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('Spring Animation')
    return fig, ax, line, line2, circle


def animate(i, x, Y_t, line,line2,circle, L=20):
    x1 = x[i]  # x-coordinate changes over time
    xs, ys = draw_spring(0, 0, x1 + L, 0.0, coils=8, amplitude=0.10, color='blue') 
    line.set_data(xs, ys)
    xs, ys = draw_spring(x1 + L, 0, Y_t[i] + 2 * L, 0.0, coils=8, amplitude=0.10, color='red') 
    line2.set_data(xs, ys)
    new_x = x1  
    circle.center = (new_x + L, 0) 
    return line, line2, circle


class DisplaceParams(NamedTuple):
  """A docstring"""
  Y0: float
  omega: float 
  phi: float = 0.0

class SHOParams(NamedTuple):
  """A docstring"""
  k: float
  m: float
  kp: float
  D: DisplaceParams
  c: float = 0.0
  initial_state: list = [0.0, 0.0]

## Exercise 1. Two springs

### a.  Equation of motion for forced displacement


 Let us now consider the scenario where we have two springs with springs constants $k$ (left) and $k'$ (right) with a mass $m$ between them.  The left spring is fixed to a wall and the right spring is displaced by a function $y(t)$.  The equation of motion for such a setup is 


$$ 
m \frac{d^2 x}{dt^2} + (k+k')x = k'y(t) 
$$

Recall that the natural frequency of this system is $\omega_0 = \sqrt{\frac{k+k'}{m}}$
In this exercise, let's generically use a displacement 

$$
y(t) = Y_0 \cos(\omega t)
$$


Work out the coupled first order differential equations and write a function `equations(t,state,params)` which returns $\dot{x}$ and $\dot{v}$.
The parameters can be set up as 
```
params = SHOParams(k=1.0, m=1.0, kp=1.0, D=DisplaceParams(Y0=4.0, omega=0.25*2*np.pi), initial_state=[1.0, 1.0])
```
which corrresponds to
* $k=k'=1.0$
* $m=1.0$
* $Y_0 = 4.0$
* $L=0.1$
* $ \omega = 0.25 (2\pi) \approx \omega_0+0.15$
* initial state of $x=1$ and $v=1$

They can be accessed like `params.kp` or `params.D.Y0`

Once you've produced this function you can go ahead and numerically compute the solution of the differential equation using 
```
sol = solve_ivp(equations, [t[0], t[-1]], params.initial_state, t_eval=t, args=(params,))
x = sol.y[0]
v= sol.y[1]
```

and plot both $x(t)$ and $Y(t)$ against $t$ for times `t = np.linspace(0, 100, 400)`. 

In order to do this, it will be useful to write a function 
```
def GetDisplacement(displaceParams, t):
### do stuff
   return Y_t
```

which you call as `GetDisplacement(params.D)` and which will return a numpy array of the displacement.


To plot them both on the same axis you can then use 
```
fig, ax1 = plt.subplots()
ax1.plot(t,x,zorder=2,label='x(t)',color='blue')
ax1.plot(t,GetDisplacement(params.D, t),zorder=1,label='y(t)',color='red',alpha=0.5)
ax1.set_xlabel("t")
ax1.set_ylabel("x(t)",color='b')
ax1.tick_params('y', colors='b')
plt.legend(loc='upper right')
plt.show()
```

You should see some clear beating going on due to being near but not on resonance. 

 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

Also go ahead and animate this system using

```
fig, ax, line, line2, circle = SetupAnimation()
ani = animation.FuncAnimation(fig, animate, frames=len(x), fargs=(x, Y_t,line,line2,circle,20), blit=True, interval=40, repeat=False)  # Pass parameters using fargs
display(HTML(ani.to_jshtml()))  
plt.close() 
```

To get the parameter `Y_t` you will need to call `Y_t=GetDisplacement(params.D0)` from above. 

 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

Finally, it's interesting to consider tuning your system actually to resonance. 

 Let $\omega$ be the natural frequency $\sqrt{(k+k')/m}$.  

 Plot $x(t)$ over time $t$ for times `t = np.linspace(0, 100, 400)`.   Here you should see the oscillations simply get larger and larger over time.  

In addition to generating this plot, let's try to figure out how large $x(t)$ gets for various values of $\omega$.  Loop over all values of $\omega$ in  `omegas=np.linspace(0.1,2,100)` and plot the largest value of $x(t)$ as a function of $\omega$.  To replace $\omega$ in your parameters you can do 

```
new_params=params._replace(D=DisplaceParams(Y0=4.0, omega=omega))
```



 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

### b.  Damping 

Our previous system was doing something unreasonable particularly as our driving frequency approached the natural frequency.  This is partially because there was no damping in the system.  Here, we will now introduce some dampling into our system.   The new equations of motion will be 

$$ 
m \frac{d^2 x}{dt^2} -cv (k+k')x = k' Y_0 \cos(\omega t)
$$

Go ahead and modify  `equations(state,t,k,kp,m,Y0,omega,c)` to solve the differential equations for this system.  Then apply it to 
`params = SHOParams(k=1.0, m=1.0, kp=1.0, c=0.5, D= D=DisplaceParams(Y0=4.0, omega=0.25*2*np.pi), initial_state=[1.0, 1.0])`


Plot $x(t)$ and animate this system out to a time of $T=20$.

 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

Now go ahead and plot the maximum $x(t)$ versus $\omega$.  You should see that the damping now prevents the maximum height you reach from diverging.  

 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

While not required for credit, it is interesting to change the relevant parameters above and put it into the stiffness controlled regime and the mass controlled regime. 

### c. $G(\omega)$

We have learned in class that the general solution of a damped driven harmonic oscillator of the form

$$
Y= F_0 \cos(\omega t+\phi)
$$ 

after an initial transient is given by 

$$
x(t) = F_0 |\tilde{G}(\omega)| \cos(\omega t + \phi + \textrm{angle}(\tilde{G}(\omega)))
$$

where the greens function 

$$
\tilde{G}(\omega) \equiv \frac{1}{k} \frac{1}{\left[1- \left(\frac{\omega}{\omega_n} \right)^2 + 2i \zeta \left(\frac{\omega}{\omega_n} \right) \right]}
$$

with 

$$
\zeta = \frac{c}{2m\omega_n}
$$


Write a function ` G(omega,k,kp,omega_n,zeta)` which computes $\tilde{G}(\omega)$. 

Then use it to plot the steady state of $x(t)$ the driven damped oscillator above on top of the exact solution as a function of time.   You expect to find that the transient roughly ends after time $2m/c$.   Plot a vertical line there on your plot - i.e. `plt.axvline(2m/c)`.

Notice that the non-transient values of $x(t)$ do not depend on the initial conditions in any way - i.e. it doesn't show up in the formula at all. 

 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

It is also interesting to look at $\tilde{G}(\omega)$ as a function of $\omega/\omega_n$.  Go ahead and make this plot both for $|\tilde{G}(\omega)|$ and $-\textrm{angle}[\tilde{G}(\omega)]$ for $\omega$ between 0 and 5. 

 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

### d.  Power Flow in the Steady State

We are interested in solving for the power flow in the steady state. The power dissipated should go as 

$$
P_\textrm{out} = cv^2 
$$

over a single cycle.

The average power input goes as 

$$
P_\textrm{in} =  Fv 
$$

Using the same parameters as before **except with $c=1$, plot both the input and output power as a function of time using `t=np.linspace(0, 20, 20000)`. 


 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>




 Now, we would like to evaluate the average power dissipated and put into the system.   We want to average the power over one (or an integer) number of cycles.

For the paramaters above, we can get the index of an integer number of wavelengths by doing
```
t_idx_start=np.argmin(np.abs(t-10.0))
t_idx_stop=np.argmin(np.abs(t-20.))
```

Go ahead and evaluate the average of the power over these times.  You can then compare it to the analytical formula 

$$
\frac{c F_0^2 \omega^2 G^2}{2}
$$

You should find that all three of these terms agree to 2-3 digits.  


 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

## Exercise 2.  Linear Combination of Periodic Forcing

### a.  Two Harmonic Forces

In this exercise, we'd like to understand what happens if we apply a driving force which is a sum of multiple Harmonic forces. 

First, let's modify the `SHOParams` to take a list of `DisplaceParams` - i.e.
```
class SHOParams(NamedTuple):
  """A docstring"""
  k: float
  m: float
  kp: float
  D: list[DisplaceParams]  
  c: float = 0.0
  initial_state: list = [0.0, 0.0]
```

Now, modify your `equations(t, state, shoParams)` to use a displacement that is the sum of the Harmonic forces (including allowing the DisplaceParams to have a offset $\phi$)

We will use initial conditions as 
```
displace1= DisplaceParams(Y0=4.0, omega=2*np.pi*0.25)
displace2= DisplaceParams(Y0=8.0, omega=0.1*2*np.pi,phi=1.0)
displaceParams= [displace1,displace2]
params = SHOParams(k=1.0, m=1.0, kp=1.0, c=0.5, D=displaceParams, initial_state=[1.0, 1.0])
```
giving us a displacement of 
$$ 
Y(t) = 4 \cos(2\pi (0.25) t) + 8 \cos( (2\pi) 0.1  t + 1.0)
$$ 

Continue using `t= np.linspace(0, 20, 10000)`

Go ahead and plot $Y(t)$ and $x(t)$.



 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

### b. Superposition of Two Harmonic Forces.  

In this section, we are going to separately solve two different Harmonic oscillators for two different forces:

**Problem 0**
$F(t) =Y(t) = 4 \cos(2\pi (0.25) t) $

Initial Conditions: [0.3,0.6]

**Problem 1**
$F(t) = + 8 \cos( (2\pi) 0.1  t + 1.0) $

Initial Conditions: [0.7,0.4]


---

Notice that the sum of the forces are the force that we had in the previous problem and the sum of the initial conditions are the initial conditions that we had in the previous problem. 

Solve for $x_0(t)$ and $x_1(t)$ seperately and plot 
* $x_0(t)$
* $x_1(t)$
* $x_0(t) + x_1(t)$
* $x(t)$ from above



 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

Suppose we had the curve `D_t=GetDisplacement(params1.D[0], t)+GetDisplacement(params2.D[1], t)` but we didn't know what frequencies it came from.  Also suppose that we had code that only solved for a single Harmonic frequency `Y_0 cos(omega t + phi)`. In such a case, we would have needed to decompose our curve into a bunch of different cosines. 

We can do this using a FFT.  Let's call 
```
fft_freqs=fftfreq(len(D_t),t[1]-t[0])
the_fft=fft(D_t)
```

Then we can plot the absolute value of $F(t)$ and this should tell us what frequencies (as a fraction of $\pi$) make up our curves
```
plt.plot(fft_freqs,np.abs(the_fft),'.')
plt.xlim(-1.2,1.2)
plt.show()
mask=np.where(np.abs(the_fft)>cutoff)
print("Frequencies: ",fft_freqs[mask])
```
These last two lines are just to indicate which frequencies are large.  You should see (both from the graph and printing) that we get back $\pm 0.25$ and $ \pm 0.1$ as large frequencies. 

To get the phases of our curves, we should then plot
```
plt.plot(fft_freqs[mask],np.angle(the_fft[mask]),'.')
plt.xlim(-1,1)
plt.ylim(-np.pi,np.pi)
plt.show()
```

 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

From these graphs above we can extract that our curve was 

$$
4.0 \exp[2 \pi i t \times 0.25] + 4.0 \exp[-2 \pi i t \times 0.25]  + 2.0 \exp[-2\pi i t \times 0.1 + 1]  + 2.0 \exp[-2 \pi i t \times 0.1 - 1]
$$


$$
= 8.0 \cos (2\pi i t \times 0.25) + 4.0 \sin (2\pi i t \times 0.1 + 1)
$$

This is what was expected given what we put in. 

### c. Harmonic Forces Giving a Square Wave

Let's now take an example of a curve that is the sum of many harmonic forces.  Let us consider a square wave generated as 

```
def GetSquareWave():
    # Parameters
    frequency =   0.2  # Frequency in Hz
    amplitude = 1
    duration = 20  # Duration in seconds
    sampling_rate = 500  # Samples per second
    duty_cycle = 0.5 # Duty cycle (0 to 1)

    # Time vector
    t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)

    # Generate the square wave
    square_wave = amplitude * signal.square(2 * np.pi * frequency * t, duty=duty_cycle)
    return square_wave,t
```

Go ahead and generate your square wave and plot it over time. 




 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

Now we'd like to take the fourier transform of the square wave.  Go ahead and do this and plot both the absolute value and angle.  



 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

At this point, we need to figure out how to take this fft information and turn it into a sum over cosines.  

Each frequency $f$ corresponds to 

$$
A \exp[2\pi i \times f] = |A|/N \cos(2\pi f + arg(A))  +  |A|/N i \sin (...)
$$

where $N$ is the sizes of the fft. 

Use this information to produce a list of displace parameters `myDisplacements` that generate a set of cosines which sum to the square wave. 

If you've done it correctly, you should get back the square wave by doing

```
myFunc=np.zeros(len(t),dtype=float)
for d in myDisplacements:
    myFunc+=GetDisplacement(d,t)
plt.plot(t,myFunc)
plt.show()
```

 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

### d.  Getting $x(t)$ from many Harmonic Forces 

 At this point, we now have a list of Harmonic displacements.  We will now generate the value of $x(t)$ in three ways.

 First, our `equations` function will just take the entire list of displacements.  Go ahead and do that and compute $x(t)$
 

 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>

Now  let's do it slightly differently.  Let's compute an $x(t)$ for each Harmonic displacement - i.e. when you call `equations` it should have only one term in the list  Then solve for each of these and sum them up to generate $x(t)$.  There is one important detail here.  You need the initial conditions to sum to `[1,1]`.  I suggest doing this by taking the first one to have that initial condition and all the others to be `[0,0]`

 <div><img src="https://clark.physics.illinois.edu/246img/AnsStart.svg" width=200 align=left alt="Answer (end)"></img><br></div>

In [ ]:
###ANSWER HERE

 <div><img src="https://clark.physics.illinois.edu/246img/AnsEnd.svg" width=200 align=left alt="Answer (end)"></img><br></div>